In [246]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, NavigableString, Comment

# AstroDienst Wiki Entry
html = urlopen('https://www.astro.com/astro-databank/Schettino,_Francesco')
bs = BeautifulSoup(html, "html.parser")
nativeDetail = bs.find('table', {'class': 'infobox'})


In [210]:
# Known By Name 🟨
flippedName = nativeDetail.find(string="Name").parent.parent.find_next_sibling('td').find('td').text
nameParts = [part.strip() for part in flippedName.split(',')]
knownByName = ' '.join(nameParts[::-1])
print(knownByName)

# Gender 🟥
gender = nativeDetail.find(string="Name").parent.parent.find_next_sibling('td').find('small').text.replace("Gender: ", "").strip()
print(gender)

# Given Names 🟨
try:
    givenName = nativeDetail.find(string="Birthname").parent.parent.find_next_sibling('td').text.split()
except AttributeError:
    givenName = knownByName
print(givenName)

# Life Event 🟦 → Birthdate 🟥
# Life Event 🟦 → Birth time 🟥
birthDateFullText = nativeDetail.find(string="born on").parent.parent.find_next_sibling('td')
birthFull = ''.join(birthDateFullText.find_all(string=True, recursive=False)).strip()
birthDate = birthFull.split(' at ')[0]
birthTime = birthFull.split(' at ')[1]
print(birthDate)
print(birthTime)

# Life Event 🟦 → Location 🟦 → Name 🟨
birthLocationFullText = nativeDetail.find(string="Place").parent.parent.find_next_sibling('td')
birthLocation = ''.join(birthLocationFullText.find_all(string=True, recursive=False)).strip().rstrip(',')
print(birthLocation)

# Life Event 🟦 (Type 🟥 = Birth) → Rodden Rating 🟦 → Data Source 🟨
dataSource = nativeDetail.find(string="Data source").parent.parent.parent.find_next_sibling('td').table.td.text
print(dataSource)

# Life Event 🟦 (Type 🟥 = Birth) → Rodden Rating 🟦 → Source Rating 🟥
sourceRoddenRating = nativeDetail.find(string="Data source").parent.parent.parent.find_next_sibling('td').table.find(string="Rodden Rating").parent.parent.parent.b.text
print(sourceRoddenRating)

Joe Lovano
M
['Joseph', 'Salvatore', 'Lovano']
29 December 1952
18:06
Cleveland, Ohio
BC/BR in hand
AA


In [214]:
# Get the sections of the AstroDienst Biography
astroDienstSections = bs.find('div', {'id': 'mw-content-text'}).find_all('h2')
for section in astroDienstSections:
    sectionType = section.get_text()
    sectionValue = []
    for sibling in section.next_siblings:
        # if it's just an empty string, like the newline character
        if isinstance(sibling, NavigableString) and sibling.strip() == '':
            continue
        # if it's a comment, which really is diagnostic information
        if isinstance(sibling, Comment):
            continue
        # if we have the native's wikimedia bio, we skip that
        if sibling.find('a',{'class':'extiw'}):
            continue
        # if we reach another headline 2, we have us a new section
        if sibling.name == 'h2':
            break
        sectionValue.append(sibling)
    print(sectionType)
    print(sectionValue)


Biography
[<p>American jazz musician, saxophone player, 1st child. One of the most mature talents operating in jazz.
</p>]
Relationships
[<ul><li>associate relationship with <a href="/astro-databank/Texier,_Henri" title="Texier, Henri">Texier, Henri</a> (born 27 January 1945)</li></ul>]
Source Notes
[<p>Steinbrecher collection: Steinbrecher has birth certificate in hand
</p>]
Categories
[<ul><li>Vocation : Entertain/Music : Instrumentalist (Saxophone player)</li></ul>]


In [215]:
# Wikimedia link to supplement
try: 
    wikimediaLink = bs.find('a', {'class': 'extiw'}).attrs['href']
except AttributeError:
    wikimediaLink = ''
print(wikimediaLink)

http://en.wikipedia.org/wiki/Joe_Lovano


## AstroDatabank image and image attribution

Find all of the images (`a.image`) on the page and grab the URLs for further exploration.

In [247]:
# Get the image links
try:
    databankImages = bs.find_all('a',{'class': 'image'})
except AttributeError:
    databankImages = []

imgURLs = []
for image in databankImages:
    imgURLs.append("https://www.astro.com" + image.attrs['href'])

print(imgURLs)


[]


In [240]:
# Get the image attribution

